# Review the pile capacity based on CPT Method

This notebook reviews the pile capacity using a CPT-based approach using different design methods.  The following assumptions will be made including

 - pile diameter = 3.5m
 - thickness of piles = 70mm
 - CPT remains constant after 60m, i.e., the termination level of the current SI Data



*TO-DOS*
- [x] process CPT data using BH08 and BH02
  - [x] filling the gaps
  - [x] extending the CPT data below 60m 
  - [x] save the modified data as json files
- [ ] calculating the effective stresses 
- [ ] calulating the shaft friction
- [ ] calculating the end bearing resistance 

In [7]:
import sys
sys.path.append(r'C:\Users\dazhong.li\OneDrive\02-Projects\01-Offshore CPT\Offshore-CPT')
from pathlib import Path
from src import CPT
from src import PipePile

ModuleNotFoundError: No module named 'src'

In [8]:
data_path = Path(r'C:\Users\dazhong.li\OneDrive\02-Projects\01-Offshore CPT\Offshore-CPT\projects\292179-Taiwan RWE\ags')

In [3]:
pile = PipePile(dia=3.5,thickness=0.07,length=70)

In [4]:
cpt_driver= CPT()
cpt_driver.read_ags(data_path /'BH08_Taichung_Rev A.ags')

list index out of range


23-02-01 14:37:12 -C:\Users\dazhong.li\OneDrive\02-Projects\01-Offshore CPT\Offshore-CPT\src\cpt.py:111 DEBUG - Import CPT Data
23-02-01 14:37:12 -C:\Users\dazhong.li\OneDrive\02-Projects\01-Offshore CPT\Offshore-CPT\src\cpt.py:293 DEBUG - Data unit is set to ['MPa', 'MPa', 'MPa']
23-02-01 14:37:12 -C:\Users\dazhong.li\OneDrive\02-Projects\01-Offshore CPT\Offshore-CPT\src\cpt.py:117 INFO - Imported data include:1834 rows and headers = ['HEADING', 'LOCA_ID', 'SCPG_TESN', 'SCPT_DPTH', 'SCPT_RES', 'SCPT_FRES', 'SCPT_PWP2', 'SCPT_SLP1', 'SCPT_SLP2', 'SCPT_FRR', 'SCPT_QT', 'SCPT_QE', 'SCPT_BDEN', 'SCPT_CPO', 'SCPT_CPOD', 'SCPT_QNET', 'SCPT_FRRC', 'SCPT_EXPP', 'SCPT_BQ', 'SCPT_ISPP', 'SCPT_NQT', 'SCPT_NFR', 'SCPT_DPPR']


In [5]:
fig = cpt_driver.plot_ags_cpt()

In [6]:
for data in fig.data:
    data.line.width=0.4
# cpt_driver.df


In [8]:
fig.write_image('BH08.pdf')

In [9]:
Path.cwd()

WindowsPath('c:/Users/dazhong.li/OneDrive/02-Projects/01-Offshore CPT/Offshore-CPT/src')

In [4]:
import numpy as np

In [5]:
depths = np.arange(0,80,0.02)

In [6]:
import pandas as pd

In [7]:
df_Fs = pd.read_csv('../RWE UWA05/Fs.csv',names=['x','y'])
df_qc = pd.read_csv('../RWE UWA05/qc.csv',names=['x','y'])
df_pwp = pd.read_csv('../RWE UWA05/PWP.csv',names=['x','y'])

In [8]:
import scipy.interpolate as interpolate

In [9]:
Fs_function = interpolate.interp1d(x=df_Fs.y, y=df_Fs.x,fill_value ='extrapolate')
qc_function = interpolate.interp1d(x=df_qc.y, y=df_qc.x,fill_value ='extrapolate')
PWP_function = interpolate.interp1d(x=df_pwp.y, y=df_pwp.x,fill_value ='extrapolate')

In [10]:
import  plotly.graph_objects as go

In [11]:
fig.add_trace(go.Scatter(x=qc_function(depths),y=depths),row=1,col=1)
fig.add_trace(go.Scatter(x=Fs_function(depths),y=depths),row=1,col=2)
fig.add_trace(go.Scatter(x=PWP_function(depths),y=depths),row=1,col=3)
fig.update_yaxes(range=[90,0])

NameError: name 'fig' is not defined

In [15]:
df_BH_08_smoothed = pd.DataFrame()

df_BH_08_smoothed['SCPT_DPTH'] = depths
df_BH_08_smoothed['SCPT_RES'] = qc_function(depths)
df_BH_08_smoothed['SCPT_FRES'] = Fs_function(depths)
df_BH_08_smoothed['SCPT_PWP2'] = PWP_function(depths)



In [16]:
df_BH_08_smoothed.to_json('df_BH_08_smoothed.json')

# Data is ready and it is time to do the calculation:

## what is requried for the pile capacity

In [105]:
import pandas as pd
CPT_smoothed = CPT()
CPT_smoothed.df = pd.read_json('./df_BH_08_smoothed.json')

In [109]:
CPT_smoothed.df.to_excel('CPT_smoothed.xlsx')

In [108]:
CPT_smoothed.set_data_unit(unit=['MPa','kPa','kPa'])
# CPT_smoothed.init_CPT()

23-02-01 18:41:11 -C:\Users\dazhong.li\OneDrive\02-Projects\01-Offshore CPT\Offshore-CPT\src\cpt.py:293 DEBUG - Data unit is set to ['MPa', 'kPa', 'kPa']


In [112]:
df = pd.read_excel(r'./CPT_smoothed_sand.xlsx')
CPT_smoothed = CPT()
CPT_smoothed.df = df
# CPT_smoothed.plot_ags_cpt()
CPT_smoothed.set_data_unit(unit=['MPa','kPa','kPa'])
CPT_smoothed.init_CPT()

C:\Users\dazhong.li\OneDrive\02-Projects\01-Offshore CPT\Offshore-CPT\src\cpt.py:592: RuntimeWarning:

divide by zero encountered in log10

C:\Users\dazhong.li\OneDrive\02-Projects\01-Offshore CPT\Offshore-CPT\src\cpt.py:558: RuntimeWarning:

invalid value encountered in log10

C:\Users\dazhong.li\OneDrive\02-Projects\01-Offshore CPT\Offshore-CPT\src\cpt.py:558: RuntimeWarning:

invalid value encountered in log10

23-02-01 18:45:24 -C:\Users\dazhong.li\OneDrive\02-Projects\01-Offshore CPT\Offshore-CPT\src\cpt.py:136 DEBUG - qt calculated using net area ratio of 0.85


In [113]:
CPT_smoothed.plot_SBTn_full()

# Set up the pile dimension for calculation of the Pile capacity

Some points to note:

1. CPT method does not explicitly consider the scour effect, as it is based on database for testing piles sometime after installation. The effect of the scour on the pile capacity shall be assessed seperately. 

In [6]:
import geoplot as plt
import utilities as ult

ModuleNotFoundError: No module named 'src'

In [119]:
pile = PipePile(dia=3.5, thickness=0.065,length=70)
pile.embedment=70

In [120]:
CPT_smoothed.shaft_friction(pile=pile,consider_clay=True)
CPT_smoothed.base_force(pile=pile)
CPT_smoothed.df['compression_capacity'] = CPT_smoothed.df['Qs'] + CPT_smoothed.df.Qb

In [121]:
fig = plt.GEOPlot.get_figure(rows=1,cols=4)
df = CPT_smoothed.df
fig.add_trace(go.Scatter(x=df['SCPT_RES'],y=df['SCPT_DPTH']),row=1,col=1)
fig.add_trace(go.Scatter(x=df['Qs'],y=df['SCPT_DPTH']),row=1,col=2)
fig.add_trace(go.Scatter(x=df['Qb'],y=df['SCPT_DPTH']),row=1,col=3)
fig.add_trace(go.Scatter(x=df['compression_capacity'],y=df['SCPT_DPTH']),row=1,col=4)
fig.update_yaxes(range=[80,0])
fig = ult.plot_showgrid(fig,naxis=4)
axis_titles = ['qc[MPa]','Qs[kPa]','Qb[kPa]','Compr.[kPa]']
ranges = [[0,30],[0,120000],[0,60000],[0,120000]]
dts = [5,20000,20000,20000]
for axis in range(1,5):
    fig.update_xaxes(side='top',title=axis_titles[axis-1],range=ranges[axis-1],dtick=dts[axis-1],row=1,col=axis)
fig.update_yaxes(title='Depth(m)')
fig.update_layout(width=1200,title=f'Pile Capacity Based on UWA05,Pile Diameter = {pile.dia_out}m, wall thickness={pile.t}m, embedment = {pile.embedment}m',margin=dict(t=200))
fig.write_image(f'Case- [{pile.dia_out}]-[{pile.t}] - [{pile.embedment}].pdf')


